In [1]:
from z3 import *

In [2]:
config_dir = "/home/ubuntu/src/lif/llvm/bench/meng/applied-crypto/3way/"
smt2_path = config_dir + "cbmc.smt2"

In [3]:
# Wrapper for allowing Z3 ASTs to be stored into Python Hashtables. 
class AstRefKey:
    def __init__(self, n):
        self.n = n
    def __hash__(self):
        return self.n.hash()
    def __eq__(self, other):
        return self.n.eq(other.n)
    def __repr__(self):
        return str(self.n)

def askey(n):
    assert isinstance(n, AstRef)
    return AstRefKey(n)

def get_vars(f):
    r = set()
    def collect(f):
      if is_const(f): 
          if f.decl().kind() == Z3_OP_UNINTERPRETED and not askey(f) in r:
              r.add(askey(f))
      else:
          for c in f.children():
              collect(c)
    collect(f)
    return {ele.n for ele in r}

def get_var_str(f):
    r = set()
    def collect(f):
      if is_const(f): 
          if f.decl().kind() == Z3_OP_UNINTERPRETED and not askey(f) in r:
              r.add(askey(f))
      else:
          for c in f.children():
              collect(c)
    collect(f)
    return {str(ele.n) for ele in r}

In [6]:
def get_cbmc_constraints_and_obsv_constraints(file_path):
    # lines = "".join(open(file_path).readlines())
    # first_line_of_obsv_constraint = "(declare-fun |Observation_0| () (_ BitVec 32))"
    smt_exprs = parse_smt2_file(file_path)
    print("Done parsing SMT2 file")
    for i in range(len(smt_exprs)):
        if "Observation_0" in str(smt_exprs[i]):
            break
    return smt_exprs[:i], smt_exprs[i:]
cbmc_constraints, obsv_constraints = get_cbmc_constraints_and_obsv_constraints(smt2_path)
print("Done splitting constraints")

cbmc_constraints_str_pair = [(c, str(c)) for c in cbmc_constraints]
obsv_constraints_str_pair = [(c, str(c)) for c in obsv_constraints]

Done parsing SMT2 file
Done splitting constraints


In [7]:
import copy
def get_dep_vars_and_cons_in_obsv(var_strs, obsv_constraints_str_pair):
    dep_vars = set()
    dep_cons = []
    for c in obsv_constraints_str_pair:
        if any(v in c[1] for v in var_strs):
            dep_vars |= get_var_str(c[0])
            dep_cons.append(c[0])
    return (dep_vars, dep_cons)


def get_dep_vars_and_cons_in_cbmc(var_strs, cbmc_constraints_str_pair):
    dep_cons = []
    dep_vars = copy.copy(var_strs)
    for c in cbmc_constraints_str_pair[::-1]:
        if any(v in c[1].split("==")[0] for v in dep_vars):
            dep_vars |= get_var_str(c[0])
            dep_cons.append(c[0])
    return (dep_vars, dep_cons)
# dep_vars_in_obsv_12921 = get_dep_vars_and_cons_in_obsv(["Observation_12921"])[0]


In [8]:
def enumerate_memeory_location_for_obsv(obsv_str, cbmc_constraints_str_pair, obsv_constraints_str_pair):
    # for c in cbmc_constraints + obsv_constraints:
    for c in obsv_constraints:
        for v in get_vars(c):
            if str(v) == obsv_str:
                obsv_var = v
                break

    solver = Solver()
    solver.reset()
    solver.set("timeout", 3*1000)
    dep_vars_in_obsv, dep_cons_in_obsv = get_dep_vars_and_cons_in_obsv([obsv_str], obsv_constraints_str_pair)
    _, dep_cons_in_cbmc = get_dep_vars_and_cons_in_cbmc(dep_vars_in_obsv, cbmc_constraints_str_pair)
    solver.add(dep_cons_in_obsv)
    solver.add(dep_cons_in_cbmc)

    # check if the premise is hard to solve. If it's the case, we relax the premise
    hard_premise = False
    for obsv_con in dep_cons_in_obsv:
        assert(is_implies(obsv_con))
        premise = obsv_con.children()[0]
        check_true = solver.check([premise == True])
        hard_premise |= check_true == unknown

    # relax the premise
    index = 0
    if hard_premise:
        new_obsv_cons = []
        new_premises = []
        for obsv_con in dep_cons_in_obsv:
            b = Bool("b" + str(index))
            index += 1
            new_premises.append(b)
            new_obsv_cons.append(Implies(b, obsv_con.children()[1]))
        new_obsv_dep_vars = set().union(*[get_var_str(c) for c in new_obsv_cons])
        _, new_cbmc_dep_cons = get_dep_vars_and_cons_in_cbmc(new_obsv_dep_vars, cbmc_constraints_str_pair)
        solver.reset()
        solver.add(new_obsv_cons + new_cbmc_dep_cons + [PbLe([(p,1) for p in new_premises], 1), PbGe([(p,1) for p in new_premises], 1)])

    # enumerate the model
    all_models = []
    for i in range(1000):
        if (solver.check() == sat):
            m = solver.model()
            all_models.append(m[obsv_var].as_signed_long())
            solver.add(obsv_var != m[obsv_var])
        elif (solver.check() == unknown):
            print("Error: Z3 timeout")
            assert(False)
        else:
            break
    if i == 999:
        print("Error: the number of models is too large")
        assert(False)

    # compute base and offset for each model
    object_id_to_offsets = {}
    object_gap = 1048576
    for val in all_models:
        object_id = val // object_gap
        if object_id not in object_id_to_offsets:
            object_id_to_offsets[object_id] = []
        object_id_to_offsets[object_id].append(val % object_gap)

    return (len(all_models), object_id_to_offsets)


# enumerate_memeory_location_for_obsv("Observation_12921", cbmc_constraints_str_pair, obsv_constraints_str_pair)

In [9]:
list_of_var_set = [get_var_str(a) for a in obsv_constraints]
list_of_vars = set().union(*list_of_var_set)
obsv_vars = [o for o in list_of_vars if "Observation_" in o]
obsv_vars.sort(key=lambda x: int(x.split("_")[-1].split("!")[0]))
len(obsv_vars)

34471

In [10]:
from joblib import Parallel, delayed


def wrapper(o, cbmc_pairs, obsv_pairs):
    return (o, enumerate_memeory_location_for_obsv(o, cbmc_pairs, obsv_pairs))
tuple_list = Parallel(n_jobs=2, require='sharedmem')(delayed(wrapper)(o, cbmc_constraints_str_pair, obsv_constraints_str_pair) for o in obsv_vars[0])
infos = dict(tuple_list)

In [ ]:
infos

In [ ]:
import re
# either a declaration, definition, lhs equality that contains var
# if none of the above, then any constraint that contains var
def is_important_constraint(constraint, var):
    if "__CPROVER_deallocated" in constraint and var in constraint:
        # we want to preserve safe-pointer constraint
        return True
    pattern = "(\(assert \(= |\(define-fun |\(declare-fun |\(assert \(= \(select )(\|.*?\|)"
    matches = re.match(pattern, constraint)
    if matches:
        if matches.group(2) == var:
            return True
        else:
            return False
    else:
        return var in constraint

[is_important_constraint("(declare-fun |Observation_2177| () (_ BitVec 32))", "|Observation_2177|"),
is_important_constraint("(define-fun |B4975| () Bool (=> |goto_symex::&92;guard#99| (not (= ((_ zero_extend 20) ((_ extract 31 20) (concat (_ bv205 12) (_ bv0 20)))) ((_ zero_extend 20) ((_ extract 31 20) |__CPROVER_deallocated#0|))))))", "|B4975|"),
is_important_constraint("(define-fun |B4975| () Bool (=> |goto_symex::&92;guard#99| (not (= ((_ zero_extend 20) ((_ extract 31 20) (concat (_ bv205 12) (_ bv0 20)))) ((_ zero_extend 20) ((_ extract 31 20) |__CPROVER_deallocated#0|))))))", "|goto_symex::&92;guard#99|"),
is_important_constraint("(assert (= |cbmc_pointer_offset_2375!0@1#2| (_ bv0 32)))", "|cbmc_pointer_offset_2375!0@1#2|"),
is_important_constraint("(assert (bvsge (bvmul |histogram::1::2::1::t!0@88#4| (_ bv4 32)) (_ bv0 32)))", "|histogram::1::2::1::t!0@88#4|"),
is_important_constraint("", "|histogram::1::2::1::t!0@88#4|"),
is_important_constraint("(assert (= |histogram::1::2::1::v!0@86#2| (select |main::1::a!0@1#1| (_ bv85 32))))", "|main::1::a!0@1#1|")]



In [ ]:
import re

def get_var_in_constraint(constraints):
    pattern = "\|.*?\||array\.\\d*"
    matches = re.findall(pattern, constraints)
    return set(matches)



def get_constraints_for_var(file_path, var_name):
    f = open(file_path)
    lines = f.readlines()
    obsv_constraints = ""
    for line in lines:
        if var_name in line:
            obsv_constraints += line
    # pattern = "\|.*?\|"
    # matches = re.findall(pattern, obsv_constraints)
    # print(matches)
    
    # dependent_vars = set(filter(lambda m: var_name not in m, matches))
    dependent_vars = get_var_in_constraint(obsv_constraints)

    lines.reverse()
    first_line_of_obsv_constraint = "(declare-fun |Observation_0| () (_ BitVec 32))"
    active_searching = False
    cbmc_constraint = ""
    for line in lines:
        if first_line_of_obsv_constraint in line:
            active_searching = True
            continue
        if active_searching and any(map(lambda dep_var: is_important_constraint(line, dep_var), dependent_vars)):
            # HACK: assert the safe-point assertion
            pattern = "\(define-fun (\|B\d*\|)"
            matches = re.match(pattern, line)
            if matches and "__CPROVER_deallocated" in line:
                cbmc_constraint = "(assert {})\n".format(matches.group(1)) + cbmc_constraint

            dependent_vars = dependent_vars | get_var_in_constraint(line)
            dependent_vars.discard("|__CPROVER_deallocated#0|")
            cbmc_constraint = line + cbmc_constraint
    print(dependent_vars)
    print("----------------")
    print(cbmc_constraint+obsv_constraints)
    print("----------------")
    return parse_smt2_string("(declare-fun |__CPROVER_deallocated#0| () (_ BitVec 32))\n" + cbmc_constraint+obsv_constraints, sorts={}, decls={})
solver.reset()
# print(*get_constraints_for_var(config_dir + "cbmc.smt2", "|Observation_12921|"), sep="\n")
get_constraints_for_var(config_dir + "cbmc.smt2", "|Observation_12921|")


In [ ]:
from z3 import *
# help_simplify()
# describe_tactics()
Tactic('cofactor-term-ite').help()

In [ ]:
from z3 import *
# solver = Then('ctx-solver-simplify', 'propagate-values', With('solve-eqs', theory_solver=False, solve_eqs_max_occs=2), 'bit-blast', 'sat-preprocess', 'sat').solver()
solver = Then('ctx-solver-simplify', 'propagate-values', With('solve-eqs', theory_solver=False, solve_eqs_max_occs=2), 'cofactor-term-ite').solver()
solver.reset()
constraints = parse_smt2_file("/tmp/12921.smt2")
solver.add(constraints)
keys = []
for a in solver.assertions():
    for v in get_vars(a):
        if "main::1::k_main!0@1#1" in str(v):
            keys.append(v)
            break
for a in solver.assertions():
    for v in get_vars(a):
        if str(v) == "Observation_12921":
            obsv = v
            break
solver.cube(keys)
print(solver.check())
m = solver.model()
print(m[obsv])
solver.statistics()

In [ ]:
print(*solver.assertions(), sep="\n")

In [ ]:
m = solver.model()
print(m[obsv])
solver.cube([obsv])
solver.add(obsv != m[obsv])
solver.check()

In [ ]:
import time


def get_differential_set_for_obsv(obsv_name):
    solver = Solver()
    solver.reset()
    start_time = time.time()
    constraints = get_constraints_for_var(config_dir+"cbmc.smt2", "|"+str(obsv_name)+"|")
    # constraints = parse_smt2_file("/tmp/cbmc.smt2", sorts={}, decls={})
    print("got constraint")
    solver.add(constraints)
    print("added constraint")

    obsv = None
    for a in solver.assertions():
        for v in get_vars(a):
            if str(v) == str(obsv_name):
                obsv = v
    assert(obsv != None)
    print("found obsv")


    slicing_elapsed_time = round(time.time() - start_time, 2)
    all_models = []
    start_time = time.time()
    for i in range(1000):
        print(i)
        if (solver.check() == sat):
            m = solver.model()
            all_models.append(m[obsv].as_signed_long())
            solver.add(obsv != m[obsv])
        else:
            break
    if i == 999:
        print("Warning: the number of models is too large")
        assert(False)
        
    elapsed_time = round(time.time() - start_time, 2)

    object_id_to_offsets = {}
    object_gap = 1048576
    for val in all_models:
        object_id = val // object_gap
        if object_id not in object_id_to_offsets:
            object_id_to_offsets[object_id] = []
        object_id_to_offsets[object_id].append(val % object_gap)
    print(obsv, slicing_elapsed_time, elapsed_time, len(constraints), len(all_models))
    return (elapsed_time, len(all_models), object_id_to_offsets)

infos = {o: get_differential_set_for_obsv(str(o)) for o in ["Observation_12921"]}


In [ ]:
from joblib import Parallel, delayed
def wrapper(o):
    return (o, get_differential_set_for_obsv(o))
tuple_list = Parallel(n_jobs=32)(delayed(wrapper)(str(o)) for o in obsvs if int(str(o).split("_")[1]) >= 9681)
infos = dict(tuple_list)

In [ ]:
infos

In [ ]:

def get_pointer_to_name_and_type_map():
    mapping = {}
    f = open(config_dir + "pointer_numbering.csv")
    for line in f.readlines():
        line = line.strip()
        splits = line.split(",")
        var_type = splits[0]
        full_var = splits[1]
        numbering = splits[2]
        #histogram::1::2::i!0@1
        #histogram$$1$$2$$1$$i
        #histogram$$1$$2$$1$$1$$t
        mapping[int(numbering)] = (full_var.split("::")[-1].split("!")[0], var_type)
    return mapping

def is_pointer_or_array(var_type):
    # return "*" in var_type or var_type.count('[') >= 2
    return var_type.count('[') >= 2

pointer_mapping = get_pointer_to_name_and_type_map()
ds_file = open(config_dir + "ds_of_obsvs.txt", "w")
for obsv, info in infos.items():
    index = str(obsv).split("_")[1]
    memory_locs = []
    ds_size = 0
    for (base, offsets) in info[2].items():
        if (base == -1):
            # decoy access is not part of ds
            continue
        var_name = pointer_mapping[base][0]
        var_type = pointer_mapping[base][1]
        for offset in offsets:
            memory_locs.append("((char*){}{})+{}".format("" if is_pointer_or_array(var_type) else "&", var_name, offset))
            ds_size += 1
    memory_locs = ",".join(memory_locs)
    memory_locs = "{"+memory_locs+"}"
    ds_file.write("#define ds_{} (void* [{}]){}\n".format(index, ds_size, memory_locs))
    ds_file.write("#define ds_size_{} {}\n".format(index, ds_size))
ds_file.write("#define ds_nil (void* []){}\n")
ds_file.write("#define ds_size_nil 0\n")
ds_file.close()

In [ ]:
start_time = time.time()
solver.check()
elapsed_time = time.time() - start_time
print("TIME: " + str(elapsed_time))

In [ ]:
bv_solver = Then(With('simplify', mul2concat=True),
                 'solve-eqs', 
                 'bit-blast', 
                 'aig',
                 'sat').solver()
start_time = time.time()
solve_using(bv_solver, solver.assertions())
elapsed_time = time.time() - start_time
print("TIME: " + str(elapsed_time))

In [ ]:
len(solver.assertions())

In [ ]:
print([obsv != m[obsv] for obsv in obsvs][:10])

In [ ]:
t = Then('simplify', 'bit-blast', 'tseitin-cnf')
g = Goal()

a, b = BitVecs('a b', 8)

g.add(Or(a == 1, a == 0))
g.add(Implies(a == 0, And(0 <= b, b < 8)))
bitmap = {}
for ob in [a,b]:
    for i in range(ob.size()):
        bitmap[(ob, i)] = Bool(str(ob)+str(i))
        mask = BitVecSort(ob.size()).cast(math.pow(2, i))
        g.add(bitmap[(ob, i)] == ((ob & mask) == mask))
sg = t(g)


In [ ]:
x, y = BitVecs('x y', 10)
s = Solver()
s.add(x > 0, y > 0, x + y == 512)
s.check()


In [ ]:
with open("/tmp/tmp.smt2", mode='w') as f:
    f.write(s.to_smt2())

In [ ]:
list_of_var_set = [get_vars(a) for a in g]
list_of_vars = set().union(*list_of_var_set)


goal = sg[0]
var_set, var_map = var_set_and_map(goal)
cnf_str = "p cnf {} {}\n".format(len(var_set), len(goal))

sampling_set = []
for ob in [a,b]:
    for i in range(ob.size()):
        sampling_set.append(var_map[bitmap[(ob, i)]])
cnf_str += "c ind {} 0\n".format(" ".join(sampling_set))


for i in range(len(goal)):
    if (is_or(goal[i])):
        for j in range(goal[i].num_args()):
            assert(goal[i].arg(j).num_args() <= 1)
            lit = goal[i].arg(j)
            if is_not(lit):
                cnf_str += "-" + var_map[lit.arg(0)]
            else:
                cnf_str += var_map[lit]
            cnf_str += " "
    else:
        # this clause is just a literal
        assert(goal[i].num_args() <= 1)
        lit = goal[i]
        if is_not(lit):
            cnf_str += "-" + var_map[lit.arg(0)]
        else:
            cnf_str += var_map[lit]
        cnf_str += " "
    cnf_str += "0\n"
    
with open("/home/cream/toy/cnf.dimacs", "w") as f:
    f.write(cnf_str)

In [ ]:
# count num of variables
def var_set_and_map(goal):    
    list_of_var_set = [get_vars(a) for a in goal]
    var_set = set().union(*list_of_var_set)

    # give each variable a name
    var_map = {}
    for i, v in enumerate(var_set):
        var_map[v] = str(i+1)

    return var_set, var_map

def generate_cnf(goal, obs):
    var_set, var_map = var_set_and_map(goal)
    cnf_str = "p cnf {} {}\n".format(len(var_set), len(goal))
    
    # gather sampling set(bits of the interested values)
    sampling_set = []
    for ob in obs:
        for i in range(ob.size()):
            sampling_set.append(var_map[bitmap[(ob, i)]])
    cnf_str += "c ind {} 0\n".format(" ".join(sampling_set))


    for i in range(len(goal)):
        if (is_or(goal[i])):
            for j in range(goal[i].num_args()):
                assert(goal[i].arg(j).num_args() <= 1)
                lit = goal[i].arg(j)
                if is_not(lit):
                    cnf_str += "-" + var_map[lit.arg(0)]
                else:
                    cnf_str += var_map[lit]
                cnf_str += " "
        else:
            print(goal[i])
            # this clause is just a literal
            assert(goal[i].num_args() <= 1)
            lit = goal[i]
            if is_not(lit):
                cnf_str += "-" + var_map[lit.arg(0)]
            else:
                cnf_str += var_map[lit]
            cnf_str += " "
        cnf_str += "0\n"
    return cnf_str

In [ ]:
with open("/home/cream/play/cnf", "w") as f:
    f.write(cnf_str)

In [ ]:
t = Then('simplify', 'bit-blast', 'tseitin-cnf')
g = Goal()

F = BitVecVal(0, 2)
T = BitVecVal(1, 2)
N = BitVecVal(2, 2)

o0, o1 = BitVecs('o0 o1', 2)
A, B, k, p, o2, o3, o4 = BitVecs('A B k p o2 o3 o4', 32) # bitvectors are signed, so use 4 instead 3 to hold a maximum value of 8

obs = [o0, o2, o3]

# secret value
g.add(0 <= k, k <= 2**3)
# public value
g.add(p == 0)
# memory allocation
g.add(A == 0, B == 0)

g.add(o0 == If(k < 4, T, F))
g.add(o1 == If(k < 4, If(k < 2, T, F), N))
g.add(o2 == If(k < 4, If(k < 2, B + p, B + p), B + p))
g.add(o3 == If(k < 4, If(k < 2, A + k / 3, A + k % 2), A + k / 4))


g.add(o0 == If(k < 4, T, F))
g.add(o2 == If(k < 4, A + k % 2, A + k /2))
g.add(o3 == B + k / 5)

obs = [o0, o2, o3, o4]
g.add(o0 == If(k < 2**19, T, N))
g.add(o2 == If(k < 2**19, A + k % 2, A + k /2))
g.add(o3 == If(k < 2**19, A + p, 2**20+1))
g.add(o4 == B + k / 5)


bitmap = {}
for ob in obs:
    for i in range(ob.size()):
        bitmap[(ob, i)] = Bool(str(ob)+str(i))
        mask = BitVecSort(ob.size()).cast(math.pow(2, i))
        g.add(bitmap[(ob, i)] == ((ob & mask) == mask))

sg = t(g)
print(g)

In [ ]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y)
print(s.check())
m = s.model()
print(m)
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

In [ ]:
solver = z3.Solver()
solver.reset()

constraints = z3.parse_smt2_file("/home/cream/src/smtapproxmc/temp_amc/temp_0_0.smt2", sorts={}, decls={})
solver.add(constraints)
solver.check()

# remove variables from CPROVER initializer
constraints = list(filter(lambda c: not(is_eq(c) and str(c.arg(0)).startswith("__CPROVER")), constraints))
print("\n------\n".join([str(c) for c in constraints]))

In [ ]:
solver.check()

In [ ]:
solver.reset()

solver.add(constraints)
solver.add(o != 7168)
solver.check()

# m = solver.model()
# list_of_var_set = [get_vars(a) for a in constraints]

# list_of_vars = set().union(*list_of_var_set)

# list_of_obsvs = [v for v in list_of_vars if str(v).startswith("Observation_")]
# # list_of_obsvs = [v for v in list_of_vars if str(v).startswith("function#return_value!0#1")]

# o = [o for o in list_of_obsvs if str(o) == "Observation_2"][0]
# m.evaluate(o)

In [ ]:
list_of_var_set = [get_vars(a) for a in constraints]

list_of_vars = set().union(*list_of_var_set)

list_of_obsvs = [v for v in list_of_vars if str(v).startswith("Observation_")]
# list_of_obsvs = [v for v in list_of_vars if str(v).startswith("function#return_value!0#1")]

list_of_obsvs

In [ ]:
len(list_of_obsvs)

In [ ]:
def transitively_select_constraints(constraints, interested_vars):

    def intersection(s0, s1):
        result = set()
        for o0 in s0:
            for o1 in s1:
                if o0.get_id() == o1.get_id():
                    result.add(o0)
        return result

    interested_vars = list_of_obsvs
    constraints = constraints_back

    new_added = True
    transitively_included_vars = set()
    transitively_included_vars.update(list_of_obsvs)
    transitively_included_constraints = set()

    constraints = set(constraints)
    constraint_var_map = {}

    # get variables used in each constraints
    for constraint in constraints:
        constraint_var_map[constraint] = get_vars(constraint)

    while new_added:
        constraints_to_add = []
        for constraint in constraints:
            if intersection(constraint_var_map[constraint], transitively_included_vars):
                constraints_to_add.append(constraint)
        new_added = False 
        for constraint in constraints_to_add:
            transitively_included_vars.update(constraint_var_map[constraint])
            transitively_included_constraints.add(constraint)
            constraints.remove(constraint)
            new_added = True
    print(transitively_included_vars)
    print(transitively_included_constraints)
    return transitively_included_constraints

selected_constraints = transitively_select_constraints(constraints, list_of_obsvs)

In [ ]:
len(selected_constraints)

In [ ]:
t = Then('simplify', 'bit-blast', 'tseitin-cnf')
g = Goal()
g.add(constraints)

bitmap = {}
for ob in list_of_obsvs:
    for i in range(ob.size()):
        bitmap[(ob, i)] = Bool(str(ob)+str(i))
        mask = BitVecSort(ob.size()).cast(math.pow(2, i))
        g.add(bitmap[(ob, i)] == ((ob & mask) == mask))
        
sg = t(g)
# print(sg[0])
cnf_str = generate_cnf(sg[0], list_of_obsvs)
cnf_str
# def equality_substitution(old_goal, new_goal):
#     e = old_goal.as_expr()
#     old_constraints = [e.arg(i) for i in range(e.num_args())]
#     equality_constraints = [c for c in old_constraints if (not is_or(c)) and c.num_args() != 1]
#     subs = []
#     for cons in equality_constraints:
# #         print(cons)
#         assert(cons.num_args() == 0 or is_eq(cons))
#         if is_eq(cons):
#             subs.append((cons.arg(0), cons.arg(1)))
#     print(subs)
#     for cons in old_constraints:
#         new_goal.add(substitute(cons, subs))
        
# # Remove equality of arrays, which can be generated from memcpy
# new_goal = Goal()
# equality_substitution(sg[0], new_goal)
# sim_sg = Tactic('simplify')(new_goal)
# cnf_str = generate_cnf(sim_sg[0], list_of_obsvs)

In [ ]:
with open("/home/cream/toy/toy.smt", "w") as f:
    f.write(cnf_str)